In [1]:
import os, sys, glob, csv
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
import xlsxwriter
import random
from random import shuffle
from math import log, floor
import re
import collections
from collections import Counter
import string
import unicodedata as udata
# import pause, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from distutils.dir_util import copy_tree
import sklearn
from sklearn.metrics import *
import itertools as it
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import functools
import spacy

import tensorflow.keras.preprocessing.text as T

stop words

In [11]:
stop_words_1 = (stopwords.words('english')) #Stopword
with open('data/preprocess/stop_words.txt') as f:
    stop_words_2 = f.read().splitlines() #stop_list1
stop_words_3 = pickle.load(open('data/preprocess/stop_list2.pkl','rb'))
stop_words_all = set(stop_words_1 + stop_words_2 + stop_words_3)
len(stop_words_all) # stopwords#

594

In [86]:
sp_tokenizer = spacy.load("en")
kk = sp_tokenizer("It's (beautiful) 4toys doesn't")
token_li=[]
for token in kk:
    token_li.append(token.text)
token_li

['It', "'s", '(', 'beautiful', ')', '4toys', 'does', "n't"]

Prepare preprocess

In [141]:
sp_tokenizer = spacy.load("en")
dil= r"[!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~]+\ *" 
dil_filter = '!\"#$%&()*+,-./:;<=>?@[\\]^_\`{|}~\t\n'
glove = pd.read_table('./data/preprocess/glove.840B.300d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
glove_words = set(glove.index.tolist())
len(glove_words)

2196009

以下deprecated

In [142]:
def text_preprcess(text,tokenizers='nltk'):
    '''
    Input: string
    Return: preprocessed list
    '''
    tokens = [i for i in text.split() if i not in stop_words_all]
    tokens_str = ' '.join(tokens)
    if tokenizers == 'keras':
        tokens = T.text_to_word_sequence(tokens_str,filters=dil_filter) # lower casse & tokenize
        tokens = list(filter(None, tokens))
    elif tokenizers == 'nltk':
#         tokens = re.sub(dil," ",tokens_str.lower())
        tokens = [i for i in tokens_str.lower() if i not in dil_filter]
        tokens = ''.join(tokens)
        tokens = word_tokenize(tokens)
        tokens = list(filter(None, tokens))
#         tokens = [x.lower() for x in tokens]
    elif tokenizers == 'spacy':
#         tokens = re.sub(dil," ",tokens_str.lower())
        tokens = [i for i in tokens_str.lower() if i not in dil_filter]
        tokens = ''.join(tokens)
        tokens = sp_tokenizer(tokens)
        token_li = []
        for token in tokens:
            token_li.append(token.text)
        tokens = list(filter(None, token_li))
#         tokens = [x.lower() for x in token_li]
    tokens_li = []
    for token in tokens:
        tokens_li.append(''.join([i for i in token if not i.isdigit()]))
    tokens_str = ' '.join(tokens_li)
    tokens_str = re.sub(dil," ",tokens_str.lower())
    tokens_li = tokens_str.split()
    tokens_li = list(filter(None, tokens_li))
    ori_tokens_num = len(tokens_li)
    tokens_li = [i for i in tokens_li if i in glove_words]
    try:
        rate = len(tokens_li)/ori_tokens_num
    except ZeroDivisionError:
        rate = np.nan
    return tokens_li , rate#ori_tokens_num/len(tokens_li)
        
    

In [143]:
# train_df = pd.DataFrame(columns=['id','category','text','original_tokenize_num','preprocess_text','preprocss_tokenize_num'])
train_df = pd.DataFrame(columns=['id','category','text','original_tokenize_num',
                                 'text_keras','rate_keras','text_nltk','rate_nltk','text_apacy','rate_spacy'])
train_dir = './data/20news-bydate-v3/20news-bydate-train/'
cat_dir = next(os.walk(train_dir))[1]
count = 0
for cat in tqdm(cat_dir):
    in_dir = train_dir + cat + '/'
    news_list = next(os.walk(in_dir))[2]
    news_list = [os.path.join(in_dir, f) for f in news_list]
    news_list = list(filter(lambda f: f.endswith(".txt"), news_list))
    for news_path in news_list:
        id_num = news_path.split('/')[-1].split('.')[0]
        with open(news_path,'r',encoding='latin1') as f:
            news_text = f.read()
            news_text = re.sub('\n',' ',news_text)
            news_text = re.sub('\t',' ',news_text)
            news_text_tok_ori = T.text_to_word_sequence(news_text,filters='',lower=False)
            news_text_ori = " ".join(news_text_tok_ori)
            news_text_tok_num_ori = len(news_text_tok_ori)
            news_text_tok_keras,rate_keras = text_preprcess(news_text,tokenizers='keras')
            news_text_tok_nltk,rate_nltk = text_preprcess(news_text,tokenizers='nltk')
            news_text_tok_spacy,rate_spacy = text_preprcess(news_text,tokenizers='spacy')
        train_df.loc[count] = [id_num,cat,news_text_ori,news_text_tok_num_ori,
                               news_text_tok_keras,rate_keras,news_text_tok_nltk,rate_nltk,news_text_tok_spacy,rate_spacy]
        count+=1
#         break
    
# train_df.loc[0] = [0,0,0]
print(train_df.rate_keras.mean() , train_df.rate_nltk.mean(),train_df.rate_spacy.mean()  )
train_df

100%|██████████| 20/20 [03:53<00:00, 11.71s/it]

0.9595051529116455 0.959558811893717 0.9605178211130393


,id,category,text,original_tokenize_num,text_keras,rate_keras,text_nltk,rate_nltk,text_apacy,rate_spacy
0,19305,rec.sport.hockey,My votes (FWIW): Team MVP: Pat Verbeek. He fan...,453,"[my, votes, fwiw, team, mvp, pat, he, fans, go...",0.950207,"[my, votes, fwiw, team, mvp, pat, he, fans, go...",0.950207,"[my, votes, fwiw, team, mvp, pat, he, fans, go...",0.950207
1,19313,rec.sport.hockey,Kovalev is too talented a player to play for R...,160,"[kovalev, talented, player, play, roger, niels...",0.951220,"[kovalev, talented, player, play, roger, needs...",0.938272,"[kovalev, talented, player, play, roger, needs...",0.938272
2,19122,rec.sport.hockey,New Jersey 1 0 2--3 Pittsburgh 2 3 1--6 First ...,919,"[new, jersey, pittsburgh, first, period, pitts...",0.867299,"[new, jersey, pittsburgh, first, period, pitts...",0.867299,"[new, jersey, pittsburgh, first, period, pitts...",0.867299
3,19312,rec.sport.hockey,"First of all, the Penguins WILL win the cup ag...",66,"[first, all, penguins, will, win, cup, again, ...",0.981481,"[first, all, penguins, will, win, cup, again, ...",0.981481,"[first, all, penguins, will, win, cup, again, ...",0.981481
4,19115,rec.sport.hockey,You can't. But good luck trying.,6,"[you, can't, but, good, luck, trying]",1.000000,"[you, ca, n't, but, good, luck, trying]",1.000000,"[you, ca, n't, but, good, luck, trying]",1.000000
...,...,...,...,...,...,...,...,...,...,...
11309,20797,alt.atheism,: When the object of their belief is said to b...,269,"[when, object, belief, said, perfect, make, be...",0.930233,"[when, object, belief, said, perfect, make, be...",0.978102,"[when, object, belief, said, perfect, make, be...",0.978102
11310,20537,alt.atheism,Koff! You mean that as long as I put you to sl...,23,"[koff, you, mean, long, i, sleep, first, i, ki...",1.000000,"[koff, you, mean, long, i, sleep, first, i, ki...",1.000000,"[koff, you, mean, long, i, sleep, first, i, ki...",1.000000
11311,20757,alt.atheism,": Mr Connor's assertion that ""more complex"" ==...",135,"[mr, assertion, more, complex, later, paleonto...",0.971014,"[mr, connor, 's, assertion, more, complex, lat...",0.985915,"[mr, connor, 's, assertion, more, complex, lat...",0.985915
11312,20677,alt.atheism,(excess stuff deleted...) I know of a similar ...,139,"[excess, stuff, deleted, i, know, similar, inc...",0.987805,"[excess, stuff, deleted, i, know, similar, inc...",0.987952,"[excess, stuff, deleted, i, know, similar, inc...",0.987952


* keras tokenizer: 95.95% tokens in glove
* nltk tokenizer: 95.96% tokens in glove
* spacy tokenizer: 96.05% tokens in glove

In [144]:
train_df = pd.DataFrame(columns=['id','category','text','original_tokenize_num','preprocess_text','preprocss_tokenize_num'])
train_dir = './data/20news-bydate-v3/20news-bydate-train/'
cat_dir = next(os.walk(train_dir))[1]
count = 0
for cat in tqdm(cat_dir):
    in_dir = train_dir + cat + '/'
    news_list = next(os.walk(in_dir))[2]
    news_list = [os.path.join(in_dir, f) for f in news_list]
    news_list = list(filter(lambda f: f.endswith(".txt"), news_list))
    for news_path in news_list:
        id_num = news_path.split('/')[-1].split('.')[0]
        with open(news_path,'r',encoding='latin1') as f:
            news_text = f.read()
            news_text = re.sub('\n',' ',news_text)
            news_text_tok_ori = T.text_to_word_sequence(news_text,filters='',lower=False)
            news_text_ori = " ".join(news_text_tok_ori)
            news_text_tok_num_ori = len(news_text_tok_ori)
#             news_text_tok_pre,rate_keras = text_preprcess(news_text,tokenizers='keras')
#             news_text_tok_pre,rate_nltk = text_preprcess(news_text,tokenizers='nltk')
            news_text_tok_pre,rate_spacy = text_preprcess(news_text,tokenizers='spacy')
            news_text_pre = " ".join(news_text_tok_pre)
        train_df.loc[count] = [id_num,cat,news_text_ori,news_text_tok_num_ori,news_text_pre,len(news_text_tok_pre)]
        count+=1
        break
    
train_df

100%|██████████| 20/20 [00:00<00:00, 42.82it/s]


,id,category,text,original_tokenize_num,preprocess_text,preprocss_tokenize_num
0,19305,rec.sport.hockey,My votes (FWIW): Team MVP: Pat Verbeek. He fan...,453,my votes fwiw team mvp pat he fans goal mouth ...,229
1,21002,comp.graphics,I have posted disp135.zip to alt.binaries.pict...,1447,i posted you distribute program freely noncomm...,912
2,21586,talk.politics.guns,Why don't you call the City and ask? Oak Park ...,327,why city ask oak park illegal handgun ban well...,180
3,23146,talk.politics.misc,-> >Now let me get this straight. After a nice...,301,now let straight after nice long rant how peop...,150
4,23229,talk.politics.mideast,"NEWS YOU MAY HAVE MISSED, APR 19, 1993 Not bec...",615,news you may have missed apr not busy us media...,368
5,19586,sci.space,Is this the one that had the {wrench|pliers} f...,12,is inside recovery,3
6,22675,sci.crypt,"Hmm, followup on my own posting... Well, who c...",973,hmm followup posting well cares first let try ...,481
7,27525,comp.sys.ibm.pc.hardware,": I'm looking to buy a 17"" monitor soon, and i...",111,i 'm looking buy monitor soon i ca n't decide ...,56
8,26073,rec.sport.baseball,Methinks you recall wrong. Mitchell hit close ...,84,methinks recall wrong mitchell hit close atlan...,39
9,28486,sci.electronics,"Fortunately, wire-wrapping is a better wiring ...",125,fortunately wirewrapping better wiring techniq...,71


以上deprecated

In [ ]:
def text_preprcess2(text):
    '''
    Goal: filter out punctuations、words not in GloVe
    * cased
    * spacy
    * for RoBERTa + GloVe embedding
    Input: text string
    Return: preprcessed text string
    '''
    tokens = [i for i in text if i not in dil_filter] #濾掉符號
    tokens = sp_tokenizer(tokens)
    token_li = []
    for token in tokens:
        token_li.append(token.text)
    tokens_li = [i for i in token_li if i in glove_words] #只留有在glove的字
    tokens = list(filter(None, tokens_li))
    tokens_str = ' '.join(tokens)
    tokens_str = re.sub(dil," ",tokens_str) #在確認濾一次符號
    return tokens_str
    

In [164]:
train_df.loc[0,'text']

'My votes (FWIW): Team MVP: Pat Verbeek. He fans on 25% of goal mouth feeds, but he still has 36 goals after a terrible start and has been an examplary (sp?) team captain throughout a tough couple of seasons. Honorable mention: Nick Kypreos and Mark Janssens. Probably more appropriate in the unsung heroes category than MVP, but Kypreos (17 goals, 320+ PIM) has been the hardest working player on the team and Janssens is underrated as a defensive center and checker. I guess I place a greater emphasis on hard work than skill when determining value. Biggest surprise: Geoff Sanderson. He had 13 goals and 31 points last season as a center, then moved to left wing and has so far put up 45 goals and 80+ points. He now has a new Whaler record 21 power play goals, most all coming from the right wing faceoff circle, his garden spot. Honorable mention: Andrew Cassels and Terry Yake. The kiddie quartet of Sanderson, Poulin, Nylander, and Petrovicky have been attracting the most attention, but Casse

In [145]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [147]:
tokens_num_li = train_df.preprocss_tokenize_num.tolist()
basic_statistics(tokens_num_li)

,length
mean,144.450000
std,214.881008
mode,23.000000
min,0.000000
q1,23.750000
median,63.500000
q3,157.500000
max,912.000000
iqr,133.750000
outlier,358.125000


In [45]:
news_text_tok

['just',
 'what',
 'do',
 'gay',
 'people',
 'do',
 'that',
 'straight',
 'people',
 "don't?",
 'absolutely',
 'nothing.',
 "i'm",
 'a',
 'very',
 'straight(as',
 'an',
 'arrow),',
 '17-year',
 'old',
 'male',
 'that',
 'is',
 'involved',
 'in',
 'the',
 'bsa.',
 'i',
 "don't",
 'care',
 'what',
 'gay',
 'people',
 'do',
 'among',
 'each',
 'other,',
 'as',
 'long',
 'as',
 'they',
 "don't",
 'make',
 'passes',
 'at',
 'me',
 'or',
 'anything.',
 'at',
 'my',
 'summer',
 'camp',
 'where',
 'i',
 'work,',
 'my',
 'boss',
 'is',
 'gay.',
 'not',
 'in',
 'a',
 "'pansy'",
 'way',
 'of',
 'gay',
 '(i',
 'know',
 'a',
 'few),',
 'but',
 'just',
 "'one",
 'of',
 'the',
 "guys'.",
 'he',
 "doesn't",
 'push',
 'anything',
 'on',
 'me,',
 'and',
 'we',
 'give',
 'him',
 'the',
 'same',
 'respect',
 'back,',
 'due',
 'to',
 'his',
 'position.',
 'if',
 'anything,',
 'the',
 'bsa',
 'has',
 'taught',
 'me,',
 'i',
 "don't",
 'know,',
 'tolerance',
 'or',
 'something.',
 'before',
 'i',
 'met',
 't

In [30]:
news_list = next(os.walk(in_dir))[2]
news_list = [os.path.join(in_dir, f) for f in news_list]
news_list

['./data/20news-bydate-v3/20news-bydate-train/alt.atheism/20556.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20748.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20810.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20574.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20692.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20705.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20519.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20565.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20835.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20649.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20684.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20872.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20739.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20547.txt',
 './data/20news-byda

# TEST

In [2]:
import torch

In [3]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval() 

Using cache found in /home/leoqaz12/.cache/torch/hub/pytorch_fairseq_master


loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz from cache at /home/leoqaz12/.cache/torch/pytorch_fairseq/83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
| dictionary: 50264 types


RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          )
          (1): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=Tru

In [7]:
tokens_ori = "now let straight after nice long rant how people need personal responsibility economic social lives sudden 's radicals such me i guess responsible poor people 's lifestyles tell think poor people dumb think themselves there reasons disintegration family and support systems general nation 's poor somehow i think murphy janis the sane person 's list you want generation 's vaunted cultural revolution lasting change worse try socalled relevant values education hey like good idea time how know needed real education mean took granted the 's generation spoiled irresponsible the depression create mothers fathers determined kids want going overboard creating nation brats consider contrast famous events july apollo woodstock which group large numbers people feed reverted cultural level primitives defecation public etc and group assembled took care itself dispersed damage deaths large numbers drug problems was n't woodstock called biggest parking lot history they rejected society went nature parent 's cars"

In [8]:
# tokens_ori = train_df.loc[3,'preprocess_text']#'how\'s are you'#
tokens = roberta.encode(tokens_ori)
# assert tokens.tolist() == [0, 31414, 232, 328, 2]
# assert roberta.decode(tokens) == 'Hello world!'
print(len(tokens) , tokens)
roberta.decode(tokens)
doc = roberta.extract_features_aligned_to_words(tokens_ori)
print(len(doc))
for tok in doc:
    print('{:10}{} (...)'.format(str(tok), tok.vector[:5]))

173 tensor([    0,  8310,   905,  1359,    71,  2579,   251, 25693,   141,    82,
          240,  1081,  2640,   776,   592,  1074,  7207,   128,    29, 35842,
          215,   162,   939,  4443,  2149,  2129,    82,   128,    29, 28182,
         1137,   206,  2129,    82, 16881,   206,  1235,    89,  2188, 32654,
         8475,   284,     8,   323,  1743,   937,  1226,   128,    29,  2129,
         7421,   939,   206, 22802, 16628, 10408,   354,     5, 37091,   621,
          128,    29,   889,    47,   236,  2706,   128,    29,   748, 19264,
         4106,  7977,  9735,   464,  3007,   860, 17380,  9315,  4249,  3266,
         1265, 17232,   101,   205,  1114,    86,   141,   216,   956,   588,
         1265,  1266,   362,  4159,     5,   128,    29,  2706, 29136, 21573,
            5,  6943,  1045,  8826, 17850,  3030,  1159,   236,   164, 35912,
         2351,  1226,  5378,  2923,  1701,  5709,  3395,  1061,  1236, 21347,
         6256, 38520,  5627,  9607,    61,   333,   739,  15

In [189]:
tokens = 'I said, "hello RoBERTa."'
tokens = sp_tokenizer(tokens)
token_li = []
for token in tokens:
    token_li.append(token.text)
len(token_li) , token_li

(8, ['I', 'said', ',', '"', 'hello', 'RoBERTa', '.', '"'])

In [167]:
last_layer_features = roberta.extract_features(tokens)
print(last_layer_features.size())# == torch.Size([1, 5, 1024])

all_layers = roberta.extract_features(tokens, return_all_hiddens=True)
assert len(all_layers) == 25
assert torch.all(all_layers[-1] == last_layer_features)

ValueError: tokens exceeds maximum length: 592 > 512

In [163]:
last_layer_features.detach().numpy().shape

(1, 19, 1024)

In [168]:
from pytorch_transformers import RobertaTokenizer

In [173]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-large-mnli')

100%|██████████| 456318/456318 [00:00<00:00, 545019.70B/s]


In [178]:
len(tokenizer.encode(train_df.loc[0,'text']))

W0925 15:40:29.729804 139833327294272 tokenization_utils.py:665] Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors


590